In [ ]:
# Re-import necessary libraries after environment reset
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

# Generate synthetic classification data
X, y = make_classification(n_samples=500, n_features=5, n_informative=4, n_redundant=0,
                           random_state=42, n_clusters_per_class=1, flip_y=0.01)

# Split into initial training and remaining for SGD updates
X_train, y_train = X[:50], y[:50]
X_remaining, y_remaining = X[50:], y[50:]

# Initialize SGD classifier
sgd = SGDClassifier(loss='log_loss', learning_rate='constant', eta0=0.01, random_state=42)
sgd.partial_fit(X_train, y_train, classes=np.unique(y))

# Track accuracy on all previously seen data
seen_X = X_train.copy()
seen_y = y_train.copy()
acc_history = [accuracy_score(seen_y, sgd.predict(seen_X))]

# Iterate over remaining data, update only if accuracy on seen data does not drop
for i in range(len(X_remaining)):
    x_new = X_remaining[i:i+1]
    y_new = y_remaining[i:i+1]

    # Save old model state
    old_coef = sgd.coef_.copy()
    old_intercept = sgd.intercept_.copy()

    # Try to update
    sgd.partial_fit(x_new, y_new)

    # Evaluate on seen data
    y_pred = sgd.predict(seen_X)
    new_acc = accuracy_score(seen_y, y_pred)

    if new_acc >= acc_history[-1]:  # Pareto improvement
        seen_X = np.vstack([seen_X, x_new])
        seen_y = np.append(seen_y, y_new)
        acc_history.append(new_acc)
    else:
        # Revert update
        sgd.coef_ = old_coef
        sgd.intercept_ = old_intercept
        acc_history.append(acc_history[-1])  # accuracy unchanged

pd.DataFrame({
    "Step": range(len(acc_history)),
    "Accuracy": acc_history
})